<br>

# Notebook 2 - Crop Simulation

<hr>
This key module simulates all the possible crop cycles to  find the best crop cycle that produces maximum yield for a particular grid. During the simulation process for each grid, 365 crop cycle simulations are performed. Each simulation corresponds to cycles that start from each day of the year (starting from Julian date 0 to Julian date 365). Similarly, this process is performed by the program for each grid in the study area. 

Prepared by Geoinformatics Center, AIT
<hr>


### Google drive connection
In this step, we will connect to Google Drive service and mount the drive where we will start our PyAEZ project

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

Then, installing any additional python packages that required to run PyAEZ.
If working on your own PC/machine, these additional installation will vary depending on what is already installed in your Python library. 

In [1]:
# 'Installing neccessary packages'
# !pip install gdal
# # !pip install pyaez==2.0.0

Now, we will import the specific Python packages we need for PyAEZ.

In [2]:
'''import supporting libraries'''
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
try:
    from osgeo import gdal
except:
    import gdal
import sys

Setting the working directory -- where our PyAEZ project is located.

In [3]:
'Set the working directory'
# Replace with path to your PyAEZv2 folder
work_dir = '/Volumes/GoogleDrive/My Drive/PyAEZv2'
os.chdir(work_dir)
sys.path.append('./pyaez/')
!pwd

# Check whether the specified path exists or not
out_path = './data_output/NB2/'
isExist = os.path.exists(out_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(out_path)
   print("The new directory is created!")


/Volumes/GoogleDrive/My Drive/PyAEZv2


<hr>

## MODULE 2: CROP SIMULATION
Now, we will start executing the routines in Module 2


First, we initiate Module 2 Class instance by invoking the following commands:

In [4]:
'''importing library'''
# Import Module 2 and initate Class intance
import CropSimulation
aez = CropSimulation.CropSimulation()

# Importing UtilitiesCalc
import UtilitiesCalc
obj_utilities = UtilitiesCalc.UtilitiesCalc()


### Importing the climate dataset and the geographical data/rasters.

The package expects six climate variables, as daily or monthly observations, as Numpy arrays.
Arrays must be 3-dimensional, with the third axes containing the time dimension.
Unit of measures are expected as follows:
- Minimum temperature = Degree Celsius
- Maximum temperature = Degree Celsius
- Precipitation = Accumulated mm / day (or per month)
- Solar radiation = W/m$^2$
- Wind speed = Average m/s
- Relative humidity = Average fraction (0 to 1)

In addition to climate data, the system requires:
- A binary admin_mask, with 0 and 1 values. 0 pixels values will be not executed, while 1 pixels values will be executed
- An elevation layer
- Soil/terrain/special land cover classes
  

**All the datasets must have the same shape.**## Reading Data

In [5]:
'''reading climate data'''
# Importing the climate data
max_temp = np.load(r'./data_input/climate/max_temp.npy')  # maximum temperature
min_temp = np.load(r'./data_input/climate/min_temp.npy')  # minimum temperature
precipitation = np.load(r'./data_input/climate/precipitation.npy')  # precipitation
rel_humidity = np.load(r'./data_input/climate/relative_humidity.npy')  # relative humidity
wind_speed = np.load(r'./data_input/climate/wind_speed.npy')# wind speed measured at two meters
short_rad = np.load(r'./data_input/climate/short_rad.npy') # shortwave radiation

# Load the geographical data/rasters
mask_path = r'./data_input/LAO_Admin.tif'
mask = gdal.Open(mask_path).ReadAsArray()
elevation = gdal.Open(r'./data_input/LAO_Elevation.tif').ReadAsArray()
soil_terrain_lulc = gdal.Open(r'./data_input/LAO_soil_terrain_lulc.tif').ReadAsArray()


This section contains parameters that can be modified by the user:
- lat_min = minimum latitude of analysis
- lat_max = maximum latitude of analysis
- mask_value = the value in the admin_mask to exclude from the analysis (typically 0)
- daily = whether climate input data are daily (True) or monthly (False)

In [6]:
# Define the Area-Of-Interest's geographical extents
lat_min = 13.87
lat_max = 22.59
mask_value = 0  # pixel value in admin_mask to exclude from the analysis
daily = False #Type of climate data = True: daily, False: monthly

### Loading the imported data into the Object Class ('*aez*' Class)

In [7]:
aez.setStudyAreaMask(mask, mask_value)
aez.setLocationTerrainData(lat_min, lat_max, elevation)
if daily:
    aez.setDailyClimateData(
        min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)
else:
    aez.setMonthlyClimateData(min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)


### Setting up the crop parameter/ crop cycle parameter and soil water parameters (Mandatory)

In [8]:
# setting up the crop parameters, crop cycle and soil water parameters ***mandatory step
aez.setCropParameters(LAI=2.3, HI=0.33, legume=0, adaptability=4, cycle_len=115, D1=0.3, D2=1)

aez.setCropCycleParameters(stage_per=[16, 26, 33, 25], kc=[0.3, 1.2, 0.5], kc_all=0.85, yloss_f=[0.4, 0.4, 0.9, 0.5], yloss_f_all=1.25)

aez.setSoilWaterParameters(Sa= 100*np.ones((mask.shape)), pc=0.5)

### Setting up the thermal screening parameters (Optional)

In [13]:
# Uncomment these screening options -- depend from case-to-case
tclimate = gdal.Open("./data_output/NB1/LAO_ThermalClimate.tif").ReadAsArray() # User to change this TClimate file 

## Thermal Climate screening
aez.setThermalClimateScreening(tclimate, no_t_climate=[2, 5, 6])

## Temperature Growing Period Screening 
aez.setLGPTScreening(no_lgpt=[75,75,75], optm_lgpt=[105,105,105])

## Temperature Summation Screening 
# aez.setTSumScreening(no_Tsum=[2100, 3600], optm_Tsum=[2300, 3150])

## Temperature Profile Screening 
# aez.setTProfileScreening(no_Tprofile=[], optm_Tprofile=[])


### Adjustment for Perennial Crop (optional)

In [14]:
# If you're working with perennial crops, this adjustment is a mandatory step
# to do

## an example for sugarcane,
#aez.adjustForPerennialCrop(aLAI=70, bLAI=200, aHI=120, bHI=180)

# but we're using non-perennial crop, so we can skip this step.


### Simulate crop cycle


In [15]:
'''run simulations'''
aez.simulateCropCycle( start_doy=1, end_doy=365, step_doy=1, leap_year=False) # results are in kg / hectare

Done: 0 %
Done: 0 %
Done: 0 %
Done: 0 %
Done: 0 %
Done: 0 %
Done: 0 %
Done: 0 %
Done: 0 %
Done: 0 %
Done: 0 %
Done: 1 %
Done: 1 %
Done: 1 %
Done: 1 %
Done: 1 %
Done: 2 %
Done: 2 %
Done: 2 %
Done: 2 %
Done: 3 %
Done: 3 %
Done: 3 %
Done: 3 %
Done: 4 %
Done: 4 %
Done: 4 %
Done: 5 %
Done: 5 %
Done: 5 %
Done: 6 %
Done: 6 %
Done: 6 %
Done: 7 %
Done: 8 %
Done: 8 %
Done: 9 %
Done: 9 %
Done: 10 %
Done: 11 %
Done: 12 %
Done: 12 %
Done: 13 %
Done: 14 %
Done: 15 %
Done: 16 %
Done: 17 %
Done: 18 %
Done: 19 %
Done: 20 %
Done: 21 %
Done: 22 %
Done: 23 %
Done: 24 %
Done: 25 %
Done: 26 %
Done: 27 %
Done: 28 %
Done: 29 %
Done: 30 %
Done: 31 %
Done: 32 %
Done: 33 %
Done: 34 %
Done: 35 %
Done: 36 %
Done: 37 %


KeyboardInterrupt: 

### Maximum Attainable Yield for Rainfed and Irrigated Condition/ Optimum starting date


Yield Classification
1.   not suitable (yields between 0% and 20%)
2.   marginally suitable (yields between 20% and 40%)
3.   moderately suitable (yields between 40% and 60%)
4. suitable (yields between 60% and 80%)
5. very suitable (yields are equivalent to 80% or more of the overall maximum yield)



In [ ]:
# Now, showing the estimated and highly obtainable yield of a particular crop, results are in kg / hectare
yield_map_rain = aez.getEstimatedYieldRainfed()  # for rainfed
yield_map_irr = aez.getEstimatedYieldIrrigated()  # for irrigated

# Optimum cycle start date, the date when the highest yield are produced referenced from the start of crop cycle
starting_date = aez.getOptimumCycleStartDate()

## get classified output of yield
yield_map_rain_class = obj_utilities.classifyFinalYield(yield_map_rain)
yield_map_irr_class = obj_utilities.classifyFinalYield(yield_map_irr)
yield_map_rain_class = np.ma.masked_where(aez.im_mask==0,yield_map_rain)
yield_map_irr_class = np.ma.masked_where(aez.im_mask==0,yield_map_irr)

In [ ]:
# Read the saved GeoTIFF files
yield_map_rain = gdal.Open('./data_output/NB2/LAO_CropSuitability_rain.tif').ReadAsArray() # for rainfed
yield_map_irr = gdal.Open(
    './data_output/NB2/LAO_CropSuitability_irr.tif').ReadAsArray()  # for irrigated

# Optimum cycle start date, the date when the highest yield are produced referenced from the start of crop cycle
starting_date = gdal.Open(
    './data_output/NB2/LAO_Starting_date.tif').ReadAsArray()

# get classified output of yield
yield_map_rain_class = gdal.Open(
    './data_output/NB2/LAO_CropSuitability_rain_class.tif').ReadAsArray()
yield_map_irr_class = gdal.Open(
    './data_output/NB2/LAO_CropSuitability_irr_class.tif').ReadAsArray()

yield_map_rain = np.ma.masked_where(aez.im_mask==0, yield_map_rain)
yield_map_irr = np.ma.masked_where(aez.im_mask == 0, yield_map_irr)
starting_date = np.ma.masked_where(aez.im_mask == 0, starting_date)
yield_map_rain_class = np.ma.masked_where(aez.im_mask == 0, yield_map_rain_class)
yield_map_irr_class = np.ma.masked_where(aez.im_mask == 0, yield_map_irr_class)


In [ ]:
'''visualize result'''

plt.figure(figsize=(8,12))
plt.subplot(3, 2, 1)
plt.imshow(yield_map_rain, cmap=plt.get_cmap('tab10', 6), vmin=-0.2, vmax=5.3)
plt.title('Crop Suitability: Rainfed')
plt.colorbar(shrink=0.9)


plt.subplot(3, 2, 2)
plt.imshow(yield_map_irr, cmap=plt.get_cmap('tab10', 6), vmin=-0.2, vmax=5.3)
plt.title('Crop Suitability: Rainfed')
plt.colorbar(shrink=0.9)

plt.subplot(3,2,3)
plt.imshow(yield_map_rain_class, cmap=plt.get_cmap('tab10', 6), vmin=-0.2, vmax=5.3)
plt.title('Yield Classes: Rainfed')
plt.colorbar(shrink=0.9)


plt.subplot(3, 2,4)
plt.imshow(yield_map_irr_class, cmap=plt.get_cmap('tab10', 6), vmin=-0.2, vmax=5.3)
plt.title('Yield Classes: Irrigated')
plt.colorbar(shrink=0.9)


plt.subplot(3, 2, 5)
plt.imshow(starting_date, vmin=0)
plt.title('Optimum starting date (Julian day)')
plt.colorbar(shrink=0.9)
plt.savefig("./data_output/NB2/LAO_CropSuitability.png",bbox_inches='tight',dpi=300)
plt.show()

In [ ]:
# '''save result'''

obj_utilities.saveRaster(mask_path, './data_output/NB2/LAO_CropSuitability_rain.tif', yield_map_rain)
obj_utilities.saveRaster(mask_path, './data_output/NB2/LAO_CropSuitability_irr.tif', yield_map_irr)

obj_utilities.saveRaster(mask_path, './data_output/NB2/LAO_Starting_date.tif', starting_date)

obj_utilities.saveRaster(mask_path, './data_output/NB2/LAO_CropSuitability_rain_class.tif', yield_map_rain_class)
obj_utilities.saveRaster(mask_path, './data_output/NB2/LAO_CropSuitability_irr_class.tif',yield_map_irr_class)

<hr>

## END OF MODULE 2: CROP SIMULATION
<hr>